In [ ]:
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage


load_dotenv()

True

In [21]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2:1b")



In [ ]:
message_list = [
    SystemMessage(content="You are a helpful assistant!"),
    HumanMessage(content="What is the capital of France?"),
    AIMessage(content="THe capital of France is Paris"),
    HumanMessage(content="What is THe capital of Germary"),
    AIMessage(content="THe capital of Germany is Berlin"),
    HumanMessage(content="What is ter capital of Italy"),
]

llm.invoke(message_list)

AIMessage(content='The capital of Italy is Rome.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 57, 'total_tokens': 64, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CAUkf1Sl6RyW2RLnbFbseJGPpqc67', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8bf126d0-3684-4191-89dc-4cfbef50f0b4-0', usage_metadata={'input_tokens': 57, 'output_tokens': 7, 'total_tokens': 64, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [35]:
# template 를 사용한 방법
# template 을 사용해야 LCEL 을 사용 할수 있다.

from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Your are a helpful assistant"),
    ("human", "What is the capital of {country}")
])

chat_prompt = chat_prompt_template.invoke({"country": "France"})

print(chat_prompt)
ai_message = llm.invoke(chat_prompt)

messages=[SystemMessage(content='Your are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of France', additional_kwargs={}, response_metadata={})]


In [34]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

answer = output_parser.invoke(llm.invoke(chat_prompt_template.invoke({"country": "Korea"})))

print(answer)

The capital of South Korea is Seoul.


In [ ]:
from pydantic import BaseModel, Field

class CountryDetail(BaseModel):
    capital: str = Field(description="The capital of the country")
    population: int = Field(description="The population of the country")
    language: str = Field(description="The language of the country")
    currency: str = Field(description="The currency of the country")


structured_llm = llm.with_structured_output(CountryDetail)

country_detail_prompt = PromptTemplate(
    template="""Give following information about {country}:
        - Capital
        - Population
        - language
        - Currency

        return it in JSON format. and return the JSON dictionary only
    """,
    input_variables=["country"],
)

json_ai_message = structured_llm.invoke(country_detail_prompt.invoke({"country": "France"}))
# json 으로 변환
json_ai_message.model_dump()


{'capital': 'Paris',
 'population': 67,
 'language': 'French',
 'currency': 'Euro'}

In [55]:
# LCEL 체인
country_prompt = PromptTemplate(
    template="""Guess teh name of the country based on the follwing information
    {information}
    return the name of the country only

    """,
    input_variables=["information"],
)

country_chain = country_prompt | llm | output_parser
answer1 = country_chain.invoke({"information": "This country is very famous for its wine in Europe"})

print(answer1)

final_chain = {"country": country_chain} | chat_prompt_template

print(final_chain)

final_chain.invoke({"information": "This country is very famous for its wine in Europe"})


Italy
first={
  country: PromptTemplate(input_variables=['information'], input_types={}, partial_variables={}, template='Guess teh name of the country based on the follwing information\n    {information}\n    return the name of the country only\n\n    ')
           | ChatOllama(model='llama3.2:1b')
           | StrOutputParser()
} middle=[] last=ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Your are a helpful assistant'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='What is the capital of {country}'), additional_kwargs={})])


ChatPromptValue(messages=[SystemMessage(content='Your are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of Italy.', additional_kwargs={}, response_metadata={})])